In [1]:
!pwd

/content


In [126]:
import tensorflow as tf
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string 
import re
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [127]:
df1 = pd.read_json("/content/Sarcasm_Headlines_Dataset.json",lines=True)
df2 = pd.read_json("/content/Sarcasm_Headlines_Dataset_v2.json",lines=True)


In [128]:
df1.count()


article_link    26709
headline        26709
is_sarcastic    26709
dtype: int64

In [129]:
df2.count()

is_sarcastic    28619
headline        28619
article_link    28619
dtype: int64

In [130]:
df = pd.concat([data1,data2])
df.count()

article_link    55328
headline        55328
is_sarcastic    55328
dtype: int64

In [131]:
df = df.drop_duplicates().reset_index(drop=True)

In [132]:
df.count()

article_link    28617
headline        28617
is_sarcastic    28617
dtype: int64

In [133]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [134]:
def clean_text(text):

  text = text.lower()

  #pattern of link
  pattern = re.compile('https?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
  text = pattern.sub('', text) #replacing link with whitespace
  # text = " ".join(filter(lambda x:x[0]!='@', text.split())) #remove handles like twitter
  emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoji, symbols, flags
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  text = emoji.sub('',text)
  text = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-]", "", text) #additional special characters
  return text

In [135]:
def token_word(dframe):

  head_line = list() #new list
  lines = dframe['headline'].values.tolist() #df values to list

  for line in lines:
    line = clean_text(line) #passing line to fxn
    tokenize = word_tokenize(line)
    pure_words = [word for word in tokenize if word.isalpha()]
    stop_words = set(stopwords.words("english"))
    filtered_words = [ word for word in pure_words if not word in stop_words]
    head_line.append(filtered_words)

  return head_line

In [136]:
head_lines = token_word(df)
head_lines[:5]

[['former',
  'versace',
  'store',
  'clerk',
  'sues',
  'secret',
  'black',
  'code',
  'minority',
  'shoppers'],
 ['roseanne',
  'revival',
  'catches',
  'thorny',
  'political',
  'mood',
  'better',
  'worse'],
 ['mom',
  'starting',
  'fear',
  'sons',
  'web',
  'series',
  'closest',
  'thing',
  'grandchild'],
 ['boehner',
  'wants',
  'wife',
  'listen',
  'come',
  'alternative',
  'debtreduction',
  'ideas'],
 ['jk', 'rowling', 'wishes', 'snape', 'happy', 'birthday', 'magical', 'way']]

In [137]:
tokenizer_obj = tf.keras.preprocessing.text.Tokenizer()
tokenizer_obj.fit_on_texts(head_lines) #firring and counting word
word_index = tokenizer_obj.word_index
print(f'Unique words ', len(word_index))

Unique words  28565


In [141]:
sequences = tokenizer_obj.texts_to_sequences(head_lines)
lines_pad = tf.keras.preprocessing.sequence.pad_sequences(sequences=sequences, maxlen=25,padding='post')

In [142]:
sentiment = data['is_sarcastic'].values #extract values

In [144]:
dimen = np.arange(lines_pad.shape[0]) #shuffle
np.random.shuffle(dimen)
lines_pad = lines_pad[dimen]
sentiment = sentiment[dimen]

In [145]:
test_sample = int(0.2 * pad_sequences.shape[0])

In [146]:
x_train = lines_pad[:-test_sample]
y_train = sentiment[:-test_sample]
x_test = lines_pad[-test_sample:]
y_test = sentiment[-test_sample:]

In [147]:
print(f' x_train = {x_train.shape} y_train = {y_train.shape} x_test = {x_test.shape} y_test = {y_test.shape} ')

 x_train = (17552, 25) y_train = (17552,) x_test = (11065, 25) y_test = (11065,) 
